In [1]:
#import dependency
from bs4 import BeautifulSoup as bs
import numpy as np
import time
import pandas as pd
from tqdm import tqdm
import re
import json

# Selenium stuff
import undetected_chromedriver as uc
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# **GET ALL EBIKE LINK**

In [ ]:
page = []
bike_name = []
bike_id = []
bike_price = []
bike_brand = []
bike_color = []
bike_category = []


#open browser
driver = uc.Chrome()

#set window size
driver.set_window_size(1920, 1080)
driver.maximize_window()

#check how many pagination the page has
driver.get('https://www.bike24.com/e-bikes.html')
pagination = bs(driver.page_source, 'html.parser').find_all('a', attrs={'class':'page-pagination-item'})
pagination = int(pagination[-1].get('data-pager-idx'))


#iterate throught pagination
for i in tqdm(range(1,pagination+1)):

    driver.get(f'https://www.bike24.com/e-bikes.html?menu=1000,314&page={i}')

    try:
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#modal-root > div > div.b24-modal-new__dialog.cookie-consent-modal.js-cookie-consent-modal > div.b24-modal-new__footer > div > div.cookie-consent-modal-simple-footer__buttons-wrapper > button:nth-child(2)"))
        )
    finally:


        try:
            #get cookies modal by css selector element
            modal = driver.find_element_by_css_selector('#modal-root > div > div.b24-modal-new__dialog.cookie-consent-modal.js-cookie-consent-modal > div.b24-modal-new__footer > div > div.cookie-consent-modal-simple-footer__buttons-wrapper > button:nth-child(2)')
            #send okay return to modal
            modal.send_keys(Keys.RETURN)

            #random sleep
            time.sleep(np.random.randint(1,5))
        
        except:
            continue

        finally:

            #pick page source code
            source = driver.page_source

            #parsing into html
            source = bs(source, 'html')

            #get item list
            items = source.find_all('a', attrs={'class':'product-tile__anchor gtm-link-root'})
            # page = page + ['https://www.bike24.com/' + str(item.get('href')) for item in source.find_all('a', attrs={'class':'product-tile__anchor gtm-link-root'})

            for item in items:
                #get datalayer enhance ecommerce data
                temp = item.get('data-gtm')
                temp = json.loads(temp)
                temp = temp['ecommerce']['click']['products'][0]

                pageTemp = str('https://www.bike24.com' + str(item.get('href')))

                page.append(pageTemp)
                bike_name.append(temp['name'])
                bike_id.append(temp['id'])
                bike_price.append(temp['price'])
                bike_brand.append(temp['brand'])
                bike_color.append(temp['variant'])

            #scroll to bottom page
            driver.execute_script("window.scrollTo(0, 75);")

            #random sleep
            time.sleep(np.random.randint(5,10))

            #Create dataframe to save the scraped item
            df = pd.DataFrame({ 
                                'Page Link': page,
                                'Bike Id': bike_id,
                                'Bike Brand' : bike_brand,
                                'Bike Name': bike_name,
                                'Bike Price €': bike_price,
                                'Bike Color' : bike_color
                            })

            #update column data type
            df['Bike Price €'] = df['Bike Price €'].astype('float')
            df['Bike Id'] = df['Bike Id'].astype('int')
            
            try:
                with pd.ExcelWriter('output.xlsx', mode='a', if_sheet_exists='overlay') as writer:  
                    df.to_excel(writer, sheet_name='ebikes_list', index=False)

            except:
                with pd.ExcelWriter('output.xlsx', mode='w') as writer:  
                    df.to_excel(writer, sheet_name='ebikes_list', index=False)
                
#when done, close the driver
driver.close()

# **GET EBIKE DETAIL**

In [8]:
#open file
df = pd.read_excel('output.xlsx', sheet_name='ebikes_list')

#all detail bike
all_detail_bike = pd.DataFrame()


#open browser
driver = uc.Chrome()

#iterate throught rowxs
for i, r in tqdm(df.iterrows()):
    driver.get(r['Page Link'])

    try:
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#productDetail > div.product-detail-tab-nav > div.product-detail-tab-nav__inner > div.product-detail-tab-nav__content-wrapper.product-detail-tab-nav__list-item--4 > div > div > div.col-md-9.col-lg-8 > div > div'))
        )
    finally:


        try:
            #get cookies modal by css selector element
            modal = driver.find_element_by_css_selector('#modal-root > div > div.b24-modal-new__dialog.cookie-consent-modal.js-cookie-consent-modal > div.b24-modal-new__footer > div > div.cookie-consent-modal-simple-footer__buttons-wrapper > button:nth-child(2)')
            #send okay return to modal
            modal.send_keys(Keys.RETURN)

            #random sleep
            time.sleep(np.random.randint(1,5))
        
        except:
            continue

        finally:


            #cms content
            # cms_content = driver.find_element_by_css_selector('#productDetail > div.product-detail-tab-nav > div.product-detail-tab-nav__inner > div.product-detail-tab-nav__content-wrapper.product-detail-tab-nav__list-item--4 > div > div > div.col-md-9.col-lg-8 > div > div')

            #random sleep
            time.sleep(np.random.randint(1,5))
            
            #scroll to middle page
            # driver.execute_script("arguments[0].scrollIntoView()", cms_content)

            #random sleep
            # time.sleep(np.random.randint(1,5))

            #scroll to middle page
            # driver.execute_script("arguments[0].scrollIntoView()", cms_content)

            #scroll 1/5 of page
            driver.execute_script("window.scrollTo(0, (document.body.scrollHeight)/5);")
            driver.execute_script("window.scrollTo(0, (document.body.scrollHeight)/5);")
            driver.execute_script("window.scrollTo(0, (document.body.scrollHeight)/5);")

            # fact_sheet = driver.find_element_by_css_selector('#productDetail > div.product-detail-tab-nav > div.product-detail-tab-nav__inner > div.product-detail-tab-nav__list-wrapper > ul > li.product-detail-tab-nav__list-item.product-detail-tab-nav__list-item--3 > label')
            fact_sheet = driver.find_element(by=By.CSS_SELECTOR, value='#productDetail > div.product-detail-tab-nav > div.product-detail-tab-nav__inner > div.product-detail-tab-nav__list-wrapper > ul > li.product-detail-tab-nav__list-item.product-detail-tab-nav__list-item--3 > label')

            WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#productDetail > div.product-detail-tab-nav > div.product-detail-tab-nav__inner > div.product-detail-tab-nav__list-wrapper > ul > li.product-detail-tab-nav__list-item.product-detail-tab-nav__list-item--3 > label'))).click()

            fact_sheet.click()

            #pick page source code
            source = driver.page_source
            #parsing into html
            source = bs(source, 'html')
            temp_table = source.find('table', attrs={'class':'product-detail-data-sheet__table'})
            table_column = temp_table.find_all('td', attrs={'class': 'product-detail-data-sheet__cell product-detail-data-sheet__cell--title'})
            table_column = [re.sub(r':','', item.text) for item in table_column]
            table_column.append('Page Link')
            table_value = temp_table.find_all('td', attrs={'class': 'product-detail-data-sheet__cell product-detail-data-sheet__cell--value'})
            table_value = [re.sub(r':','', item.text) for item in table_value]
            table_value.append(r['Page Link'])
            detail_table = dict(zip(table_column, table_value))
            detail_table = pd.DataFrame([detail_table]) 
            all_detail_bike  = pd.concat([all_detail_bike, detail_table], ignore_index=True)
            with pd.ExcelWriter('output.xlsx', mode='a', if_sheet_exists='overlay') as writer:  
                all_detail_bike.to_excel(writer, sheet_name='ebikes_list_detail', index=False)
            #random sleep
            time.sleep(np.random.randint(1,5))         

#when done, close the driver
driver.close()

# #Create dataframe to save the scraped item
# df = pd.DataFrame({ 
#                     'Page Link': page,
#                     'Bike Id': bike_id,
#                     'Bike Brand' : bike_brand,
#                     'Bike Name': bike_name,
#                     'Bike Price €': bike_price,
#                     'Bike Color' : bike_color
#                     })

# #update column data type
# df['Bike Price €'] = df['Bike Price €'].astype('float')
# df['Bike Id'] = df['Bike Id'].astype('int')

0it [00:00, ?it/s]/var/folders/q9/dtzpz1cx4px1gz2kwh9__dyc0000gn/T/ipykernel_13427/2324699473.py:24: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  modal = driver.find_element_by_css_selector('#modal-root > div > div.b24-modal-new__dialog.cookie-consent-modal.js-cookie-consent-modal > div.b24-modal-new__footer > div > div.cookie-consent-modal-simple-footer__buttons-wrapper > button:nth-child(2)')
0it [00:13, ?it/s]


ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (213, 1001)
  (Session info: chrome=99.0.4844.74)
Stacktrace:
0   60f75fe02b81ad5c_chromedriver       0x0000000100717159 60f75fe02b81ad5c_chromedriver + 5120345
1   60f75fe02b81ad5c_chromedriver       0x00000001006a4b13 60f75fe02b81ad5c_chromedriver + 4651795
2   60f75fe02b81ad5c_chromedriver       0x0000000100294e68 60f75fe02b81ad5c_chromedriver + 392808
3   60f75fe02b81ad5c_chromedriver       0x00000001002d17e8 60f75fe02b81ad5c_chromedriver + 641000
4   60f75fe02b81ad5c_chromedriver       0x00000001002cf288 60f75fe02b81ad5c_chromedriver + 631432
5   60f75fe02b81ad5c_chromedriver       0x00000001002cca04 60f75fe02b81ad5c_chromedriver + 621060
6   60f75fe02b81ad5c_chromedriver       0x00000001002cb78c 60f75fe02b81ad5c_chromedriver + 616332
7   60f75fe02b81ad5c_chromedriver       0x00000001002bf109 60f75fe02b81ad5c_chromedriver + 565513
8   60f75fe02b81ad5c_chromedriver       0x000000